# Imports

In [4]:
# import plaidml.keras
# plaidml.keras.install_backend()
import wandb
import os
import multiprocessing
import rasterio
import tensorflow as tf
from glob import glob
import pickle
import numpy as np
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow_addons.metrics import F1Score, HammingLoss
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

import tensorflow.keras as keras
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

In [2]:
tf.__version__

'2.4.1'

# Test with Simple CNN and Data Loader 

In [3]:
import boto3
import rasterio as rio
import numpy as np
import io

In [4]:
from data_loader import DataLoader

In [5]:
import wandb
from wandb.keras import WandbCallback 

wandb.init(project='test', sync_tensorboard=True)
config = wandb.config

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: zwarshavsky (use `wandb login --relogin` to force relogin)
/Users/purgatorid/opt/anaconda3/envs/ml-conda/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
# Variables definition
config.batch_size = 20
config.learning_rate = 0.001
config.label_file_path_train="labels_test_v1.csv" # labels_1_4_train_v2
config.label_file_path_val="val_labels.csv"
config.loss = SigmoidFocalCrossEntropy() # tf.keras.losses.BinaryCrossentropy(from_logits=False)
config.optimizer = keras.optimizers.Adam(config.learning_rate)
config.input_shape = (100,100,18)
config.numclasses=10

config.augment = True
config.flip_left_right=True
config.flip_up_down=False
config.rot90=False
config.transpose=False
config.enable_shuffle=True

print(f"batch size {config.batch_size}, learning_rate {config.learning_rate}, augment {config.augment}")

batch size 20, learning_rate 0.001, augment True


In [7]:
gen = DataLoader(label_file_path_train=config.label_file_path_train, # test labels_test_v1 TODO use s3 paths
                label_file_path_val=config.label_file_path_val, # or val_all
                bucket_name='canopy-production-ml',
                data_extension_type='.tif',
                training_data_shape=(100, 100, 18),
                augment=config.augment,
                s3_file_paths=False,
                flip_left_right=config.flip_left_right,
                flip_up_down=config.flip_up_down,
                rot90=config.rot90,
                transpose=config.transpose,
                enable_shuffle=config.enable_shuffle,
                training_data_batch_size=config.batch_size,
                enable_data_prefetch=True
                )


The file labels_test_v1.csv is missing positive labels for classes ['0', '2', '3', '7', '8', '9']
The file val_labels.csv is missing positive labels for classes ['5', '9']
Data augmentation enabled: flip_up_down False, flip_left_right True, rot90 False
Training on 200 images
Validation on 366 images 


In [8]:
gen.class_weight_val

{0: 4.8977272727272725,
 1: 0.4081439393939394,
 2: 1.6325757575757576,
 3: 2.8355263157894735,
 4: 0.5793010752688172,
 6: 1.2827380952380951,
 7: 0.7696428571428571,
 8: 1.7379032258064515}

In [9]:
gen.class_weight_train

{1: 0.76875, 4: 0.4270833333333333, 5: 10.25, 6: 3.84375}

In [10]:
def plot_learningCurve(history):
  # Plot training & validation accuracy values
  plt.plot(history.history['accuracy'])
#   plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
#   plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train'], loc='upper left')
  plt.show()
  
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Val'], loc='upper left')
  plt.show()

  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Val'], loc='upper left')
  plt.show()

# Preproduction Candidate: ResNet50 pretrained on ImageNet

In [11]:
def define_model(numclasses,input_shape):
    # parameters for CNN
    input_tensor = Input(shape=input_shape)

    # introduce a additional layer to get from bands to 3 input channels
    input_tensor = Conv2D(3, (1, 1))(input_tensor)

    base_model_resnet50 = keras.applications.ResNet50(include_top=False,
                              weights='imagenet',
                              input_shape=(100, 100, 3))
    base_model = keras.applications.ResNet50(include_top=False,
                     weights=None,
                     input_tensor=input_tensor)

    for i, layer in enumerate(base_model_resnet50.layers):
        # we must skip input layer, which has no weights
        if i == 0:
            continue
        base_model.layers[i+1].set_weights(layer.get_weights())

    # add a global spatial average pooling layer
    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)

    # let's add a fully-connected layer
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
    # and a logistic layer
    predictions = Dense(numclasses, activation='sigmoid')(top_model)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
#     model = model.layers[-1].bias.assign([0.0]) # WIP getting an error ValueError: Cannot assign to variable dense_8/bias:0 due to variable shape (10,) and value shape (1,) are incompatible

#     model.summary()
    return model

In [12]:
from data_loader import SaveCheckpoints

random_id = random.randint(1,10001) 
base_name_checkpoint = "model_resnet"
save_checkpoint_s3 = SaveCheckpoints(base_name_checkpoint)

# model_checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(
#   filepath= f'checkpoints/checkpoint_loss_{random_id}.h5',
#   format='h5',
#   verbose=1,
#   save_weights_only=True,
#   monitor='val_loss',
#   mode='min',
#   save_best_only=True)

# model_checkpoint_callback_recall = tf.keras.callbacks.ModelCheckpoint(
#   filepath= f'checkpoints/checkpoint_recall_{random_id}.h5',
#   format='h5',
#   verbose=1,
#   save_weights_only=True,
#   monitor='val_recall',
#   mode='max',
#   save_best_only=True)

# model_checkpoint_callback_precision = tf.keras.callbacks.ModelCheckpoint(
#   filepath= f'checkpoints/checkpoint_precision_{random_id}.h5',
#   format='h5',
#   verbose=1,
#   save_weights_only=True,
#   monitor='val_precision',
#   mode='max',
#   save_best_only=True)

# reducelronplateau = tf.keras.callbacks.ReduceLROnPlateau(
#   monitor='val_loss', factor=0.1, patience=5, verbose=1,
#   mode='min', min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_precision', mode='max', patience=20, verbose=1)

wandb_callback = WandbCallback()
callbacks_list = [save_checkpoint_s3,wandb_callback,early_stop] #TODO re add reducelronplateau

model = define_model(config.numclasses, config.input_shape)

# loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), # Computes the cross-entropy loss between true labels and predicted labels.
# Focal loss instead of class weights: https://www.dlology.com/blog/multi-class-classification-with-focal-loss-for-imbalanced-datasets/
model.compile(loss=SigmoidFocalCrossEntropy(), # https://www.tensorflow.org/addons/api_docs/python/tfa/losses/SigmoidFocalCrossEntropy
              optimizer=keras.optimizers.Adam(config.learning_rate),
              metrics=[tf.metrics.BinaryAccuracy(name='accuracy'), 
                       tf.keras.metrics.Precision(name='precision'), # Computes the precision of the predictions with respect to the labels.
                       tf.keras.metrics.Recall(name='recall'), # Computes the recall of the predictions with respect to the labels.
                       F1Score(num_classes=10, name="f1_score") # https://www.tensorflow.org/addons/api_docs/python/tfa/metrics/F1Score
                       ]
#               sample_weight_mode="temporal" # This argument is not supported when x is a dataset or a dataset iterator, instead pass sample weights as the third element of x.
             )



In [ ]:
epochs = 2
history = model.fit(gen.training_dataset, validation_data=gen.validation_dataset, 
                    epochs=epochs, 
                    callbacks=callbacks_list,
                    shuffle=True # whether to shuffle the training data before each epoch
                   ) 


Epoch 1/2
10/10 [==============================] - 51s 4s/step - loss: 1.5361 - accuracy: 0.8211 - precision: 0.2587 - recall: 0.2177 - f1_score: 0.0807 - val_loss: 4.5341 - val_accuracy: 0.8822 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00

Epoch 0 saving checkpoint
Epoch 2/2


KeyboardInterrupt: 

# Evaluation

In [33]:
loss, accuracy, precision, recall, f1score = model.evaluate(gen.validation_dataset) 

 1/19 [>.............................] - ETA: 38s - loss: 2.1143 - accuracy: 0.9550 - precision: 0.9500 - recall: 0.7037 - f1_score: 0.0974

KeyboardInterrupt: 

In [ ]:
s3 = boto3.resource('s3')
# TODO test on entire test dataset
obj = s3.Object('canopy-production-ml', "chips/cloudfree-merge-polygons/split/test/100/100_1000_1000.tif")
obj_bytes = io.BytesIO(obj.get()['Body'].read())
with rasterio.open(obj_bytes) as src:
    img_test = np.transpose(src.read(), (1, 2, 0))
print(img_test.shape)
label_list = ['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
    'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
    'Water'
]

Exception: The wandb backend process has shutdown

# Sandbox

In [67]:
def csv_to_tfrecord(csv_filename,out_path="test_dataset.tfrecords"):

    def create_tf_example(label, filename):

        tf_example = tf.train.Example(features=tf.train.Features(feature={
                    'Labels':tf.train.Feature(float_list=label),
                    'Filename':tf.train.Feature(bytes_list=filename),
        }))
        return tf_example

    csv = pd.read_csv(csv_filename).values
    with tf.io.TFRecordWriter(out_path) as writer:
        for row in csv:
            label_raw, filename_str = row[:-1], row[-1]
            label = tf.train.FloatList(value=labels)
            filename = filename_str.encode(encoding='UTF-8',errors='strict')
            filename = tf.train.BytesList(value=[filename])
            example = create_tf_example(label, filename)
            writer.write(example.SerializeToString())
        writer.close()

In [68]:
csv_to_tfrecord("labels_train.csv",out_path="labels_train.tfrecords")

In [69]:
csv_to_tfrecord("labels_val.csv",out_path="labels_val.tfrecords")